In [ ]:
import os
import sys
import update_prices_functions as upf
import send_message as sm

In [ ]:
import configparser

config_file_name = 'price_monitoring.config'
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
database = cfg['database_config']['database']

In [ ]:
#URL SAMPLE LIST TO TEST
url_list = [
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/5835511110?isAddedCart=',
    r'https://www.coupang.com/vp/products/6060667011?isAddedCart=',
    r'https://www.coupang.com/vp/products/2530237?isAddedCart=',
    r'https://www.coupang.com/vp/products/266030794?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='
]

upf.register_price_from_url_list(url_list)

In [ ]:
#UPDATE ALL PRICES LISTED INTO DATABASE
upf.update_db_prices()

In [ ]:
#SEND MESSAGE TO USERS WITH PRODUCTS WITH PRICE LOWER THAN TARGET PRICE
sm.send_message_to_users(database)